In [9]:
# Import libraries
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date as date_today
from datetime import timedelta


In [2]:
# Define a empty dataframe
df = pd.DataFrame(columns=["Title","Location","Company","Salary",'DOP',"Description",'link'])

# open chrome
driver = webdriver.Chrome("./chromedriver")

# Main loop
for i in range(0,100,10):

    #Open URL
    driver.get(f"https://www.indeed.co.in/jobs?q=Data+Scientist%2C+data+analyst&start={i}")
    # let driver wait to find eement
    driver.implicitly_wait(20)
    # filter out the jobs 
    all_jobs = driver.find_elements_by_class_name('result')

    for job in all_jobs:

        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html,'html.parser')
        
        # find the title
        try:
            title = soup.find("a",class_="jobtitle").text.replace('\n','')
        except:
            title = 'None'
        # find the location
        try:
            location = soup.find(class_="location").text
        except:
            location = 'None'
        # find the date
        try:
            date = soup.find(class_='date').text
        except:
            date = 'None'
            
            # find the company
        try:
            company = soup.find(class_="company").text.replace("\n","").strip()
        except:
            company = 'None'
        # find the salary
        try:
            salary = soup.find(class_="salary").text.replace("\n","").strip()
        except:
            salary = 'Not Disclosed'
        
        #find the job description
        sum_div = job.find_elements_by_class_name("summary")[0]
        try:
            sum_div.click()
        except:
            #Disable the popup if occur
            close_button = driver.find_elements_by_class_name("popover-x-button-close")[0]
            close_button.click()
            sum_div.click()

        # gather job description
        jd = driver.find_element_by_id('vjs-desc').text
        # find the link for applying
        
        link =job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href")
        
        df = df.append({'Title':title,'Location':location,"Company":company,
                        "Salary":salary,'DOP':date,"Description":jd,'link':link},
                        ignore_index=True)





In [3]:
df.head()

,Title,Location,Company,Salary,DOP,Description,link
0,Data Mapping Analyst,"Mumbai, Maharashtra",DATAMATRIX TECHNOLOGIES PVT LTD,"₹20,000 - ₹22,000 a month",5 days ago,--LOOKING FOR IMMEDIATE HIRE--\nThe role will ...,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
1,Financial Research Analyst,"Mumbai, Maharashtra","NPM, Inc.","₹6,00,000 - ₹7,50,000 a year",30+ days ago,"NPM, Inc.\nCompany Background\nNew Project Med...",https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
2,Scientist - Biosensors/ Gas Sensors,"Bengaluru, Karnataka","BreathX Technologies Inc, ( Canary Health Tech...","₹35,000 a month",4 days ago,Canary Health Technologies’ builds cutting edg...,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
3,Data Entry Analyst,"Pune, Maharashtra","HRTMS, Inc.",Not Disclosed,30+ days ago,Title: Data Entry Analyst\nWritten Communicati...,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
4,Data Scientist I,India,Ola Cabs,Not Disclosed,Today,About this role:\nThe Data Science & Machine L...,https://www.indeed.co.in/rc/clk?jk=1f29ab78e76...


In [23]:
# Function to show the date correctly
def date_corrector(days):
    
    if days.lower() =='today':
        days = str(date_today.today())
        
    elif days.lower() == '1 day ago':
        days = str(date_today.today() - timedelta(1))
        
    elif days.lower() == '30+ days ago':
        days = 'Posted more than a month ago'
        
    else:
        day = days.replace(' days ago','')
        days = str(date_today.today() - timedelta(int(day)))
        
    return days

In [24]:
# Function to show the job description correctly
def des_clear(des):
    return des.replace('\n',' ')

In [25]:
# Since Indeed does not provide seperate element for skills we will have to find out skills this way
def skills_finder(des):
    skills_list = ['database', 'statistic','Probability','Sales' ,'Marketing','Excel', 'English grammar','Banking','Finance', 'python','matlab','tableau','sql','azure','SAS','nlp','tensorflow','machine learning','pytorch','RNN','CNN','image processing','LSTM','AWS']
    skills_req = []
    
    for i in skills_list:
        if i.lower() in des.lower():
            skills_req.append(i)
    return ' ,'.join(skills_req)

In [26]:
def rs_to_flat(strs):
    return strs.replace('₹',' Rs')

In [31]:
df['DOP']= df.DOP.apply(date_corrector)
df.Description = df.Description.apply(des_clear)
df['Skills'] = df.Description.apply(skills_finder)
df['Salary'] = df.Salary.apply(rs_to_flat)

In [32]:
# Re arrange the Dataframe
col =['Title', 'Location', 'Company', 'Skills','Salary', 'Description', 'DOP','link']
df = df[col]

In [33]:
df.head()

,Title,Location,Company,Skills,Salary,Description,DOP,link
0,Data Mapping Analyst,"Mumbai, Maharashtra",DATAMATRIX TECHNOLOGIES PVT LTD,database,"Rs20,000 - Rs22,000 a month",--LOOKING FOR IMMEDIATE HIRE-- The role will r...,2020-09-29,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
1,Financial Research Analyst,"Mumbai, Maharashtra","NPM, Inc.","database ,Banking ,Finance","Rs6,00,000 - Rs7,50,000 a year","NPM, Inc. Company Background New Project Media...",Posted more than a month ago,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
2,Scientist - Biosensors/ Gas Sensors,"Bengaluru, Karnataka","BreathX Technologies Inc, ( Canary Health Tech...","statistic ,Excel ,matlab","Rs35,000 a month",Canary Health Technologies’ builds cutting edg...,2020-09-30,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
3,Data Entry Analyst,"Pune, Maharashtra","HRTMS, Inc.","Excel ,English grammar",Not Disclosed,Title: Data Entry Analyst Written Communicatio...,Posted more than a month ago,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6...
4,Data Scientist I,India,Ola Cabs,"statistic ,Excel ,python ,tensorflow ,machine ...",Not Disclosed,About this role: The Data Science & Machine Le...,2020-10-04,https://www.indeed.co.in/rc/clk?jk=1f29ab78e76...


In [34]:
df.to_csv("data_science_jobs_INDEED.csv",index=False)
